In [ ]:
! pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/ImageFaces/Train'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an1.jpeg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an2.jpeg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an3.jpg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an4.jpg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an5.jpeg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an7 (1).jpg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an9avif
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an8.jpeg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an9 (1).jpg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an10.jpg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an6.jpg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an8.jpg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an7.jpg
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an4.png
/content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an9.jpg
/content/dri

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Data Preparation

In [ ]:
import cv2
from mtcnn.mtcnn import MTCNN

def assure_path_exists(path):
    """Check and create the directory if it does not exist."""
    if not os.path.exists(path):
        os.makedirs(path)

def prepare_data(data_dir = '/content/drive/MyDrive/ImageFaces/Train', output_dir = '/content/drive/MyDrive/ImageFaces/training_data/', required_size=(160, 160)):
  detector = MTCNN()

  assure_path_exists(output_dir)

  # Initialize variables
  face_id = 0  # Unique ID for each person

  Y = []
  X = []
  person_ids = {}

  # Loop through each person folder
  for person_name in os.listdir(data_dir):
      person_path = os.path.join(data_dir, person_name)

      if not os.path.isdir(person_path):
          continue

      print(f"Processing images for {person_name}...")

      # Create a subdirectory for each person in the output directory
      person_output_dir = os.path.join(output_dir, person_name)
      assure_path_exists(person_output_dir)

      # Increment face ID for each person
      face_id += 1
      count = 0

      # Loop through each image of the person
      for image_name in os.listdir(person_path):
          image_path = os.path.join(person_path, image_name)

          # Read the image
          image = cv2.imread(image_path)

          if image is None:
              print(f"Could not read {image_path}. Skipping...")
              continue

          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

          # Detect faces in the image
          faces = detector.detect_faces(image)

          for face in faces:
              x, y, w, h = face['box']
              count += 1

              # Crop and save the face region
              face = image[y:y+h, x:x+w]
              face = cv2.resize(face, required_size)
              face_filename = os.path.join(person_output_dir, f"Person.{face_id}.{count}.jpg")
              cv2.imwrite(face_filename, face)
              X.append(face)
              Y.append(face_id)
              person_ids[face_id] = person_name  # Store ID-name mapping


          print(f"Processed {count} faces for {person_name}.")
  print("Processing complete.")
  return X, Y, person_ids

X, Y, person_ids = prepare_data()
#X_test, Y_test = prepare_data('/content/drive/MyDrive/ImageFaces/Val', '/content/drive/MyDrive/ImageFaces/validation_data/')

Processing images for Annalise Keating...
Processed 1 faces for Annalise Keating.
Processed 2 faces for Annalise Keating.
Processed 3 faces for Annalise Keating.
Processed 4 faces for Annalise Keating.
Processed 5 faces for Annalise Keating.
Processed 6 faces for Annalise Keating.
Could not read /content/drive/MyDrive/ImageFaces/Train/Annalise Keating/an9avif. Skipping...
Processed 7 faces for Annalise Keating.
Processed 8 faces for Annalise Keating.
Processed 9 faces for Annalise Keating.
Processed 10 faces for Annalise Keating.
Could not read /content/drive/MyDrive/ImageFaces/Train/Annalise Keating/.ipynb_checkpoints. Skipping...
Processed 11 faces for Annalise Keating.
Processed 12 faces for Annalise Keating.
Processed 13 faces for Annalise Keating.
Processed 14 faces for Annalise Keating.
Processed 15 faces for Annalise Keating.
Processed 16 faces for Annalise Keating.
Processed 17 faces for Annalise Keating.
Processing images for Manuelle...
Processed 1 faces for Manuelle.
Process

# Image embedding

In [ ]:
! pip install keras-facenet

  Preparing metadata (setup.py) ... done
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10368 sha256=3d572086655d287b8fbd6c1288db1b2a511f885c6274e0b73a978bff0b798fa0
  Stored in directory: /root/.cache/pip/wheels/99/94/dd/cb1a65a7440ba6d508bd24346c15af0b1d24ff8b1cdb1c9959
Successfully built keras-facenet


In [ ]:
from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(model, face_pixels):
	face_pixels = face_pixels.astype('float32') # 3D (160x160x3)
	face_img = np.expand_dims(face_pixels, axis=0)
  # 4D (Nonex160x160x3)
	embedding = model.embeddings(face_img)
	return embedding[0] # 512 D image (1x1x512)

In [ ]:
EMBEDDED_X = []

for face in X:
  EMBEDDED_X.append(get_embedding(embedder, face))

EMBEDDED_X = np.array(EMBEDDED_X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━

In [ ]:
np.save('/content/drive/MyDrive/ImageFaces/training_data/EMBEDDED_3classes.npz', EMBEDDED_X, Y)

# SVM model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC, OneClassSVM
from sklearn.metrics.pairwise import cosine_similarity

X_train, X_test, Y_train, Y_test = train_test_split(EMBEDDED_X, Y, test_size=0.2, random_state=42, shuffle=True)

model = SVC(kernel='linear', probability=True)
model.fit(X_train, Y_train)

# Train One-Class SVM for unknown detection
one_class_svm = OneClassSVM(gamma='auto')
one_class_svm.fit(X_train)

OneClassSVM(gamma='auto')

In [ ]:
y_preds_train = model.predict(X_train)
y_preds_test = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(Y_train, y_preds_train)
test_accuracy = accuracy_score(Y_test, y_preds_test)

In [ ]:
train_accuracy, test_accuracy

(1.0, 1.0)

# Test Pipeline

In [ ]:
detector = MTCNN()
# Recognize face and detect unknowns
def recognize_face(test_img_path, threshold=0.5):
    test_img = cv2.imread(test_img_path)
    test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

    faces = detector.detect_faces(test_img)
    if len(faces) == 0:
        return "No face detected"

    x, y, w, h = faces[0]['box']
    face_img = test_img[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (160, 160))
    embedding = get_embedding(embedder, face_img)

    # Predict class
    pred_label = model.predict([embedding])[0]
    pred_prob = np.max(model.predict_proba([embedding]))

    # Outlier detection using One-Class SVM
    is_known = one_class_svm.predict([embedding])[0]  # 1 = known, -1 = unknown

    # Distance-based rejection
    known_embeddings = X_train
    distances = np.linalg.norm(known_embeddings - embedding, axis=1)
    min_distance = np.min(distances)

    print(f"Predicted Class: {person_ids.get(pred_label, 'Unknown')}, Prob: {pred_prob:.2f}, Distance: {min_distance:.2f}")
    print(pred_prob)

    if pred_prob < threshold or min_distance > 0.3 or is_known == -1:
        return "Unknown"

    return person_ids.get(pred_label, "Unknown")

# Example recognition
test_img_path = "/content/drive/MyDrive/ImageFaces/Test/20250220104954.jpg"
recognize_face(test_img_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted Class: Manuelle, Prob: 0.96, Distance: 0.27


'Manuelle'

In [ ]:
import joblib

# Save the SVC model
joblib.dump(model, 'svc_model.pkl')

# Save the OneClassSVM model
joblib.dump(one_class_svm, 'one_class_svm_model.pkl')

['one_class_svm_model.pkl']